# Tokenizer

(samples modified from:  https://huggingface.co/docs/transformers/main/en/preprocessing)

In [38]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

In [39]:
# see: https://huggingface.co/docs/transformers/main/en/preprocessing
encoded_input = tokenizer("Do not meddle in the affairs of wizards, for they are subtle and quick to anger.")
print(encoded_input['input_ids'])
print(encoded_input['token_type_ids'])
print(encoded_input['attention_mask'])

[101, 2091, 1136, 1143, 13002, 1107, 1103, 5707, 1104, 16678, 1116, 117, 1111, 1152, 1132, 11515, 1105, 3613, 1106, 4470, 119, 102]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


# Handling multiple Sentences

In [40]:
batch_sentences = [
    "But what about second breakfast?",
    "Don't think he knows about second breakfast, Pip.",
    "What about elevensies?",
]
encoded_inputs = tokenizer(batch_sentences)

# Wir definieren eine kleine Hilfsfunktion, um die Parameter ausgeben zu können:
def outp (inputs):
    for name in ['input_ids', 'token_type_ids', 'attention_mask']:
        if name in inputs:
            print(f"\n{name}")
            for i in range (len (inputs[name])):
                print(inputs[name][i])

outp(encoded_inputs)


input_ids
[101, 1252, 1184, 1164, 1248, 6462, 136, 102]
[101, 1790, 112, 189, 1341, 1119, 3520, 1164, 1248, 6462, 117, 21902, 1643, 119, 102]
[101, 1327, 1164, 5450, 23434, 136, 102]

token_type_ids
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]

attention_mask
[1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1]


# Padding & Truncation

In [41]:
encoded_inputs = tokenizer(batch_sentences, padding=True, truncation=True)

In [42]:
outp(encoded_inputs)


input_ids
[101, 1252, 1184, 1164, 1248, 6462, 136, 102, 0, 0, 0, 0, 0, 0, 0]
[101, 1790, 112, 189, 1341, 1119, 3520, 1164, 1248, 6462, 117, 21902, 1643, 119, 102]
[101, 1327, 1164, 5450, 23434, 136, 102, 0, 0, 0, 0, 0, 0, 0, 0]

token_type_ids
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

attention_mask
[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]


# Word based tokenizer

In [43]:
text = 'But what about second breakfast?'
tokenizer.tokenize(text)

['But', 'what', 'about', 'second', 'breakfast', '?']

# Sub-Word based tokenizer

In [44]:
from transformers import XLNetTokenizer

tokenizer = XLNetTokenizer.from_pretrained("xlnet/xlnet-base-cased")
output = tokenizer.tokenize("Don't you love 🤗 Transformers? We sure do.")
for i in range (len (output)):
    print(output[i], end=' | ')

▁Don | ' | t | ▁you | ▁love | ▁ | 🤗 | ▁ | Transform | ers | ? | ▁We | ▁sure | ▁do | . | 

- Transformers wurde in zwei Sub-Words aufgeteilt
- Satzzeichen habe eigene Tokens

# Your own Tokenizer

## Einen Textcorpus laden
[Credits to huggingface: https://github.com/huggingface/notebooks/blob/main/examples/tokenizer_training.ipynb]

## Ein Dataset von huggingface laden:
Wir laden zunächst das "Wikitext" Dataset von huggingface. Darin enthalten sind Beispieltexte von Wikipedia.

In [45]:
from datasets import load_dataset

In [46]:
dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="train")

# Zugriffsmöglichkeiten auf das DataSet

In [47]:
dataset

Dataset({
    features: ['text'],
    num_rows: 36718
})

In [48]:
dataset[30000]

{'text': " Le souper de Beaucaire was a political pamphlet written by Napoleon Bonaparte in 1793 . With the French Revolution into its fourth year , civil war had spread across France between various rival political factions . Napoleon was involved in military action , on the government 's side , against some rebellious cities of southern France . It was during these events , in 1793 , that he spoke with four merchants from the Midi and heard their views . As a loyal soldier of the Republic he responded in turn , set on dispelling the fears of the merchants and discouraging their beliefs . He later wrote about his conversation in the form of a pamphlet , calling for an end to the civil war . \n"}

In [49]:
dataset[30000:30005]

{'text': [" Le souper de Beaucaire was a political pamphlet written by Napoleon Bonaparte in 1793 . With the French Revolution into its fourth year , civil war had spread across France between various rival political factions . Napoleon was involved in military action , on the government 's side , against some rebellious cities of southern France . It was during these events , in 1793 , that he spoke with four merchants from the Midi and heard their views . As a loyal soldier of the Republic he responded in turn , set on dispelling the fears of the merchants and discouraging their beliefs . He later wrote about his conversation in the form of a pamphlet , calling for an end to the civil war . \n",
  '',
  ' = = Background = = \n',
  '',
  ' During the French Revolution the National Convention became the executive power of France , following the execution of King Louis XVI . With powerful members , such as Maximilien Robespierre and Georges Danton , the Jacobin Club , a French political

# DataSet für den Trainnigsprozess bereitstellen (Corpus)
## A) als Liste von Listen
Dies lässt sich einfach umsetzen, hat aber den Nachteil, dass die Daten im Hauptspeicher gehalten werden müssen.

In [50]:
batch_size = 1000
all_texts = [dataset[i : i + batch_size]["text"] for i in range(0, len(dataset), batch_size)]

## B) Als Iterator
Ein (Python-) Iterator ist eine Möglichkeit, Daten sukzessive nach und nach zu laden. Da das DataSet die Daten auf der Festplatte vorhält, wird für den Iterator immer nur ein Teil der Daten in den Hauptspeicher geladen.

Wir definieren diesen hier als Funktion:

In [51]:
def batch_iterator():
    for i in range(0, len(dataset), batch_size):
        yield dataset[i : i + batch_size]["text"]

# Verschiedene Tokenizer aufbauen
# 1) Die Architektur eines vorhandenen Tokenizers clonen aber selbst trainieren
[Credits to: https://github.com/huggingface/notebooks/blob/main/examples/tokenizer_training.ipynb]

In [52]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Wir stellen sicher, dass wir eine "fast" Version der Architektur geladen haben, um das Training im Rahmen zu halten
if not tokenizer.is_fast:
    raise ("Dieser Tokenizer ist nicht geeignet.")
else:
    print("Dieser Tokenizer ist geeignet.")

Dieser Tokenizer ist geeignet.


In [53]:
gpt_clone_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(), vocab_size=25000)

In [54]:
outp(gpt_clone_tokenizer(dataset[30000:30005]["text"]))


input_ids
[826, 17937, 536, 416, 1298, 13209, 5762, 321, 259, 2140, 23294, 1680, 370, 12929, 19894, 284, 22994, 272, 1648, 261, 1613, 5537, 650, 520, 2321, 607, 264, 2720, 1123, 461, 4425, 1959, 2319, 771, 2026, 5417, 2140, 17597, 272, 12929, 321, 2820, 284, 1743, 2492, 264, 326, 261, 1465, 331, 83, 1615, 264, 935, 829, 9740, 695, 4828, 281, 2260, 2319, 272, 593, 321, 724, 1084, 2051, 264, 284, 22994, 264, 358, 394, 7996, 359, 847, 18439, 403, 261, 6486, 73, 288, 5902, 507, 6462, 272, 725, 259, 12325, 10998, 281, 261, 2932, 394, 5882, 284, 1470, 264, 890, 326, 3004, 2122, 261, 12300, 281, 261, 18439, 288, 1070, 11706, 286, 507, 9621, 272, 555, 787, 1346, 756, 405, 9631, 284, 261, 803, 281, 259, 23294, 264, 4336, 337, 389, 789, 290, 261, 2720, 1123, 272, 315]
[]
[301, 301, 5074, 301, 301, 315]
[]
[1366, 261, 1613, 5537, 261, 1518, 14686, 937, 261, 5518, 1443, 281, 2319, 264, 1212, 261, 10053, 281, 1300, 4460, 17075, 41, 272, 1648, 3990, 1781, 264, 844, 344, 19096, 306, 3788, 3691, 1350

### Diesen (trainierten!) Tokenizer können wir auch zur Wiederverwertung speichern:

In [56]:
gpt_clone_tokenizer.save_pretrained("../data/gpt_clone_tokenizer")

('../data/gpt_clone_tokenizer\\tokenizer_config.json',
 '../data/gpt_clone_tokenizer\\special_tokens_map.json',
 '../data/gpt_clone_tokenizer\\vocab.json',
 '../data/gpt_clone_tokenizer\\merges.txt',
 '../data/gpt_clone_tokenizer\\added_tokens.json',
 '../data/gpt_clone_tokenizer\\tokenizer.json')

## 2) Architektur eines Tokenizers selbst aufbauen
### Beispiel: Byte Pair Encodings (wie für GPT-2)

In [58]:
# Benötigte Imports
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer

## Tokenizer-Pipeline
"Um zu verstehen, wie man einen Tokenizer von Grund auf erstellt, müssen wir ein wenig mehr in die Tokenizer-Bibliothek und die Tokenisierungs-Pipeline eintauchen. Diese Pipeline besteht aus mehreren Schritten:

* **Normalisierung**: Führt alle anfänglichen Transformationen über die anfängliche Eingabezeichenkette aus. Wenn Sie z.B. einen Text klein schreiben, ihn vielleicht entfernen oder einen der üblichen Unicode-Normalisierungsprozesse anwenden wollen, fügen Sie einen Normalizer hinzu.
* **Pre-Tokenizer**: Verantwortlich für die Aufteilung der ursprünglichen Eingabezeichenfolge. Das ist die Komponente, die entscheidet, wo und wie die ursprüngliche Zeichenkette vorsegmentiert wird. Das einfachste Beispiel wäre, einfach an Leerzeichen zu trennen.
* **Modell**: Übernimmt die gesamte Erkennung und Generierung von Sub-Token. Dieser Teil kann trainiert werden und ist wirklich von den Eingabedaten abhängig.
* **Post-Processing**: Bietet erweiterte Konstruktionsfunktionen, um mit einigen der Transformers-basierten SoTA-Modelle kompatibel zu sein. Für BERT wird der tokenisierte Satz beispielsweise um [CLS]- und [SEP]-Token "verpackt"."

Vgl. https://github.com/huggingface/notebooks/blob/main/examples/tokenizer_training.ipynb

### Ein BPE-Tokenizer benötigt einen Pre-Tokenizer (word based tokens)

In [59]:
# Basismodell ist ein BPE-Tokenizer
tokenizer = Tokenizer(models.BPE())

In [66]:
# BPE Byte Level Pre-Tokenizer werden über huggingface zur Verfügung gestellt.
tokenizer.pre_tokenizer.pre_tokenize_str("Tokenizer sind toll!")
# add_prefix_space=False verhindert, dass vor dem ersten Wort ebenfalls das Trennzeichen 'Ġ' erscheint. Alle weiteren Token werden mit diesem Zeichen eingeleitet.
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)


In [67]:
# Ergebnisse des Pre-Tokenizers anziegen
tokenizer.pre_tokenizer.pre_tokenize_str("Tokenizer sind toll!")

[('Tokenizer', (0, 9)),
 ('Ġsind', (9, 14)),
 ('Ġtoll', (14, 19)),
 ('!', (19, 20))]

### Unseren Tokenizer trainieren
Der Tokenizer muss trainiert werden, denn er muss die Byte Pairs lernen.

In [68]:
trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

### Post-Prozessing
Der Tokenizer erhält noch ein Post-Prozessing.
Der ByteLevel PostProcessor kümmert sich um das Trimming der Offsets. Whitespaces können optional ebenfalls getrimmt werden.


In [69]:
tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
tokenizer.decoder = decoders.ByteLevel()

### Tokenizer für Huggingface Framework "verpacken"
HuggingFace stellt Wrapper zur Verfügung, die sicherstellen, dass die Tokenizer zu den APIs der Modelle passen.

Hier verwenden wir einen GPT2TokenizerFast.


In [72]:
from transformers import GPT2TokenizerFast

custom_gpt_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer)

### Neuen Tokenizer anwenden

In [75]:
outp(custom_gpt_tokenizer(dataset[30000:30005]["text"]))


input_ids
[763, 17874, 473, 353, 1235, 13146, 5699, 258, 196, 2077, 23231, 1617, 307, 12866, 19831, 221, 22931, 209, 1585, 198, 1550, 5474, 587, 457, 2258, 544, 201, 2657, 1060, 398, 4362, 1896, 2256, 708, 1963, 5354, 2077, 17534, 209, 12866, 258, 2757, 221, 1680, 2429, 201, 263, 198, 1402, 268, 83, 1552, 201, 872, 766, 9677, 632, 4765, 218, 2197, 2256, 209, 530, 258, 661, 1021, 1988, 201, 221, 22931, 201, 295, 331, 7933, 296, 784, 18376, 340, 198, 6423, 73, 225, 5839, 444, 6399, 209, 662, 196, 12262, 10935, 218, 198, 2869, 331, 5819, 221, 1407, 201, 827, 263, 2941, 2059, 198, 12237, 218, 198, 18376, 225, 1007, 11643, 223, 444, 9558, 209, 492, 724, 1283, 693, 342, 9568, 221, 198, 740, 218, 196, 23231, 201, 4273, 274, 326, 726, 227, 198, 2657, 1060, 209, 252]
[]
[238, 238, 5011, 238, 238, 252]
[]
[1303, 198, 1550, 5474, 198, 1455, 14623, 874, 198, 5455, 1380, 218, 2256, 201, 1149, 198, 9990, 218, 1237, 4397, 17012, 41, 209, 1585, 3927, 1718, 201, 781, 281, 19033, 243, 3725, 3628, 1287,